### IMPORT LIBRARIES

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os
import tqdm
import glob
import tensorflow
import keras

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.applications import VGG16,EfficientNetB2,ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/qubvel/efficientnet.git

### IMPORT / VIEWING / PREPROCESSING DATASET

> `DATA AUGMENTATION`

In [ ]:
data_augmentation = keras.Sequential(
    [   keras.layers.experimental.preprocessing.RandomZoom(-0.3,-0.2),
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,

                                   )



test_datagen  = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = '/content/drive/MyDrive/BC_IDC/train',
                                                   target_size = (512,512),
                                                   class_mode = 'categorical',

                                                   batch_size = 8)

In [ ]:
test_dataset = test_datagen.flow_from_directory(directory = '/content/drive/MyDrive/BC_IDC/test',
                                                  target_size = (512,512),
                                                  class_mode = 'categorical',

                                                  batch_size = 8)

### MODEL BUILDING

In [ ]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


In [ ]:
# Model Initialization
import keras

modelpath = '/content/drive/MyDrive/ResNet50pretrained_imagnet_finetuning_weights.h5'
base_model = keras.models.load_model(modelpath)
base_model.summary()


In [ ]:

#base_model.get_layer(name='resnet50').name = 'resnet50'
input  = keras.Input(shape=(512,512,3))
x = data_augmentation(input)
for layer in base_model.layers[0].layers[1:]:
  x = layer(x)
for layers in base_model.layers[1:-1]:
  x = layer(x)

In [ ]:


base_model.summary()

In [ ]:
def change_model(model, new_input_shape=(None, 40, 40, 3),custom_objects=None):
    # replace input shape of first layer

    config = model.layers[0].layers[0].get_config()
    config['batch_input_shape']=new_input_shape
    model.layers[0]._layers[0]=model.layers[0].layers[0].from_config(config)

    # rebuild model architecture by exporting and importing via json
    new_model = tensorflow.keras.models.model_from_json(model.to_json(),custom_objects=custom_objects)

    # copy weights from old model to new one
    for layer in new_model._layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {}".format(layer.name))
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    return new_model
new_model = change_model(base_model, new_input_shape=[None] + [512,512,3])

In [ ]:
base_model = keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(512, 512, 3),
    include_top=False)

In [ ]:
for layer in new_model.layers[0].layers[:143]:
  layer.trainable = False


In [ ]:
base_model.layers[0].summary()

In [ ]:
model = keras.models.Sequential()
model.add(new_model)
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3,activation='softmax'))

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.


model.summary()

In [ ]:
inputs = keras.Input(shape=(512, 512, 3))
x = data_augmentation(inputs)
x = base_model(x)
x = keras.layers.Flatten()(x)
x =  Dropout(0.5)(x)
outputs = keras.layers.Dense(3,activation='softmax')(x)
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:




# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x)
x = Flatten()(x)
x = Dense(64,kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5,name = 'dropout_1')(x)
x = Dense(64,kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5,name = 'dropout_2')(x)
x = Dense(64,kernel_initializer='he_uniform')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5,name = 'dropout_3')(x)
outputs = Dense(3,activation='softmax')(x)

model = keras.Model(input, outputs)

model.summary()


In [ ]:
# Freezing Layers

for layer in base_model.layers:
    layer.trainable=False

In [ ]:
base_model.summary()

In [ ]:
# Building Model

model=Sequential()
model.add(base_model)
model.add(Dense(64,kernel_initializer='he_uniform'))
x=model.add(BatchNormalization())
x=model.add(Activation('relu'))
x=model.add(Dropout(0.5))
x=model.add(Dense(64,kernel_initializer='he_uniform'))
x=model.add(BatchNormalization())
x=model.add(Activation('relu'))
x=model.add(Dropout(0.5))
x=model.add(Dense(64,kernel_initializer='he_uniform'))
x=model.add(BatchNormalization())
x=model.add(Activation('relu'))
x=model.add(Dropout(0.5))
x=model.add(Dense(3,activation='softmax'))

In [ ]:
# Summary

model.summary()

In [ ]:
# Model Compile

OPT    = tensorflow.keras.optimizers.Adam(lr=0.001)
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc')
]

base_model.compile(loss='categorical_crossentropy',
              metrics= METRICS,
              optimizer=OPT)

In [ ]:
# Defining Callbacks

filepath = '/content/drive/MyDrive/ResNet50pretrained2_imagnet_finetuning_weights.h5'

earlystopping = EarlyStopping(monitor = 'val_accuracy',
                              mode = 'max' ,
                              patience = 60,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath,
                                monitor = 'val_accuracy',
                                mode='max',
                                save_best_only=True,
                                verbose = 1)
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 5,verbose = 1,factor = 0.75, min_lr = 1e-10)


callback_list = [earlystopping, checkpoint, lrd]

In [ ]:
model_history=base_model.fit(train_dataset,
                        validation_data=test_dataset,
                        epochs = 100,
                        callbacks = callback_list,
                        verbose = 1)

### MODEL EVALUATION

In [ ]:
# Summarize history for loss

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [ ]:
# Summarize history for loss

plt.plot(model_history.history['auc'])
plt.plot(model_history.history['val_auc'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [ ]:
# Improting Image class from PIL module
from PIL import Image

# Opens a image in RGB mode
#im = Image.open(r"/content/drive/MyDrive/BC_IDC/train/G1/01_BC_G1_9057_10x_1.JPG")

# importing the module
import cv2
from google.colab.patches import cv2_imshow
import glob
PATH = '/content/drive/MyDrive/BC_IDC/40x/train_original/G1_cropped'
for filename in glob.glob('/content/drive/MyDrive/BC_IDC/40x/train_original/G1/*.*'):

  img=cv2.imread(filename, cv2.IMREAD_UNCHANGED)





  height,width=img.shape[:2]
  cv2_imshow(img)


  start_row,start_col=int(width*0),int(height*0.1)
  end_row,end_col=int(width*1),int(height*1)


  cropped=img[start_row:-1,start_col:-50]

  cv2.imwrite(os.path.join(PATH , filename), cropped)
  cv2_imshow(cropped)



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

img = cv2.imread('/content/drive/MyDrive/BC_IDC/40x/train_40x/G1/01_BC_G1_9057_40x_1_JPG.rf.3b7f25e2099e04523f2d671b37ee8202.jpg', cv2.IMREAD_UNCHANGED)
#cv2.putText(img,"original image")
cv2_imshow(img)

In [ ]:
%matplotlib

In [ ]:
%matplotlib inline

In [ ]:
from PIL import Image
from io import BytesIO
PATH = '/content/drive/MyDrive/BC_IDC/40x/train_original/G3'
for filename in glob.glob('/content/drive/MyDrive/BC_IDC/40x/train_original/G3/*.*'):


    img = Image.open(filename)

    w,h = img.size
    (left, upper, right, lower) = (w*0.08, h*0.066, w*0.84, h*1)
    im_crop  = img.crop((left, upper, right, lower))
    im_crop.save(os.path.join(PATH,filename))







**Testing deepslide framework on this dataset**

**DeepSlide: A Sliding Window Framework for Classification of High Resolution Microscopy Images**
[github repo link](https://github.com/BMIRDS/deepslide.git)


In [ ]:
!git clone https://github.com/BMIRDS/deepslide.git

In [ ]:
%cd /content/deepslide

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python code/4_test.py --all_wsi  /content/drive/MyDrive/20x/20x_all \
               --auto_select False \
               --preds_test   /content/drive/MyDrive/20x/pred_test \
               --preds_val /content/drive/MyDrive/20x/pred_val \
               --num_layers 18 \
               --patches_eval_val /content/drive/MyDrive/20x/val_patches \
               --patches_eval_test /content/drive/MyDrive/20x/test_patches \
               --checkpoints_folder /content/drive/MyDrive/checkpoints/20x_checkpoint/65 \
               --checkpoint_file   resnet18_e65_va0.66948.pt \
               --num_workers 8 --batch_size 256

In [ ]:



!python code/5_grid_search.py --all_wsi  /content/drive/MyDrive/20x/20x_all \
               --preds_val /content/drive/MyDrive/20x/pred_val \
               --inference_val   /content/drive/MyDrive/20x/inference_val


In [ ]:
!python code/6_visualize.py --all_wsi  /content/drive/MyDrive/20x/20x_all \
               --wsi_val /content/drive/MyDrive/20x/valid_20x \
               --preds_val /content/drive/MyDrive/20x/pred_val \
               --wsi_test  /content/drive/MyDrive/20x/test_20x \
               --preds_test /content/drive/MyDrive/20x/pred_test \
               --vis_val /content/drive/MyDrive/20x/vis_val \
               --vis_test /content/drive/MyDrive/20x/vis_test


In [ ]:
!python code/7_final_test.py --all_wsi /content/drive/MyDrive/20x/20x_all \
--inference_val /content/drive/MyDrive/20x/inference_val \
--labels_val /content/drive/MyDrive/20x/labels/valid_labels_20x.csv \
--labels_test /content/drive/MyDrive/20x/labels/test_labels_20x.csv \
--preds_test /content/drive/MyDrive/20x/pred_test \
--inference_test /content/drive/MyDrive/20x/inference_test

In [ ]:
--all_wsi /content/drive/MyDrive/BC_IDC/40x/40x_all \
                    --inference_val  /content/drive/MyDrive/BC_IDC/40x/inference_val \
                    --labels_val /content/drive/MyDrive/BC_IDC/40x/labels/valid_labels.csv \
                    --preds_test /content/drive/MyDrive/BC_IDC/40x/pred_test_csv \
                    --inference_test /content/drive/MyDrive/BC_IDC/40x/inference_test \
                    --labels_test  /content/drive/MyDrive/BC_IDC/40x/labels/test_labels.csv

In [ ]:
import os
import csv
img_to_label = []
header = ['img', 'gt']
for root, dirs, files in os.walk('/content/drive/MyDrive/BC_IDC/40x/test_40x'):

  for name in files:
    label = os.path.join(root,name).split('/')[-2]
    img_name = os.path.join(root,name).split('/')[-1].split('.')[0] + '.jpg'
    rows = [img_name,label]
    img_to_label.append(rows)


with open('/content/drive/MyDrive/BC_IDC/40x/labels/test_labels.csv','w') as label_file:
      csv_writer = csv.writer(label_file)
      csv_writer.writerow(header)
      csv_writer.writerows(sorted(img_to_label))

In [ ]:
!python /content/deepslide/code/1_split.py  --all_wsi  40x_all/ \
                         --labels_test labels/labels_test.csv \
                         --labels_train labels/labels_train.csv \
                         --labels_val labels/labels_val.csv \
                         --wsi_test ex_test/ \
                         --wsi_train ex_train/ \
                         --wsi_val ex_val/

In [ ]:
%cd /content/drive/MyDrive/BC_IDC/40x

In [ ]:

with open('/content/drive/MyDrive/BC_IDC/40x/labels/valid_labels.csv','r') as lines_open:

        lines = lines_open.readlines()[1:]

        file_to_gt_label = {}

        for line in lines[0:3]:
            if len(line) > 3:
                pieces = line[:-1].split(",")
                print(pieces)
                file = pieces[0]
                gt_label = pieces[1]

                file_to_gt_label[file] = gt_label
                print(file_to_gt_label)


